#Part A

##Libraries

The **materialLibrary** contains the value of convective and conductive unit thermal resistance (R-value) for some materials in specified length in [mm] (in the case of conductive thermal resistance).

*NOTE: In using this library, pay attention to the length of the layer and the standard length of corrosponding R_value*

In [49]:
materialLibrary = {
    "gypsumBoard":{"standardLength":13, "Rvalue":0.079},
    "woodFiberBoard":{"standardLength":13, "Rvalue":0.23},
    "woodBevelLappedSiding":{"standardLength":13, "Rvalue":0.14},
    "woodStud":{"standardLength":90, "Rvalue":0.63},
    "glassFiber":{"standardLength":25, "Rvalue":0.7},
    "outsideAir":{"Rvalue":0.03},
    "insideAir":{"Rvalue":0.12} 
}

In defining the **airGapLibrary**, we assumed the position of air gap is vertical and the direction of heat flow is horizental.
Furthurmore, the mean temperature and the temperature difference between the surfaces of air gap is assumed as 10 and 5.6 degreeC, respectively.

With the help of this dictionary, we can extract the radiative R-value of an airgap, by knowing the effective emissivity, for airgap spaces of 20, 40, and 90 mm.

In [50]:
airGapLibrary = {
    20:{0.03:0.65, 0.05:0.61, 0.5:0.25, 0.82:0.18},
    40:{0.03:0.67, 0.05:0.62, 0.5:0.26, 0.82:0.18},
    90:{0.03:0.64, 0.05:0.60, 0.5:0.25, 0.82:0.18}
}

##Calculation of overall unit thermal resistance & heat transfer coefficient

In [51]:
def effectiveEmissivity (eps1=0.9, eps2=0.9):
  """This function calculate effective emissivity and both epsilon values are defined as 0.9 by default."""
  result = round(1/(1/eps1 + 1/eps2 - 1), 2)
  return result

In [52]:
R1 = {"name":"outsideAir", "type":"conv"}
R2 = {"name":"woodFiberBoard", "type":"cond", "length":13}
R3 = {"name":"woodBevelLappedSiding", "type":"cond", "length":13}
R4 = {"name":"glassFiber", "type":"cond", "length":90}
R5 = {"name":"woodStud", "type":"cond", "length":90}
R6 = {"name":"gypsumBoard", "type":"cond", "length":13}
R7 = {"name":"insideAir", "type":"conv"}
R8 = {"name":"airGap", "type":"rad", "gapLength":40, "eps2":0.05}

In [53]:
resistanceList = [R1, R2, R3, R4, R5, R6, R7, R8]

In [54]:
i = 0
for anyR in resistanceList:
  i +=1
  if anyR["type"] == "cond":
    for anyMember in materialLibrary:
      if anyR["name"] == anyMember:
        anyR["Rvalue"] = round(materialLibrary[anyMember]["Rvalue"]*anyR["length"]/materialLibrary[anyMember]["standardLength"],4)
  elif anyR["type"] == "conv":
    for anyMember in materialLibrary:
      if anyR["name"] == anyMember:
        anyR["Rvalue"] = materialLibrary[anyMember]["Rvalue"]
  elif anyR["type"] == "rad":
    effectiveEmissivityCalc = effectiveEmissivity(eps2=anyR["eps2"])
    anyR["Rvalue"] = airGapLibrary[anyR["gapLength"]][effectiveEmissivityCalc]


In [55]:
resistanceList

[{'Rvalue': 0.03, 'name': 'outsideAir', 'type': 'conv'},
 {'Rvalue': 0.23, 'length': 13, 'name': 'woodFiberBoard', 'type': 'cond'},
 {'Rvalue': 0.14,
  'length': 13,
  'name': 'woodBevelLappedSiding',
  'type': 'cond'},
 {'Rvalue': 2.52, 'length': 90, 'name': 'glassFiber', 'type': 'cond'},
 {'Rvalue': 0.63, 'length': 90, 'name': 'woodStud', 'type': 'cond'},
 {'Rvalue': 0.079, 'length': 13, 'name': 'gypsumBoard', 'type': 'cond'},
 {'Rvalue': 0.12, 'name': 'insideAir', 'type': 'conv'},
 {'Rvalue': 0.62,
  'eps2': 0.05,
  'gapLength': 40,
  'name': 'airGap',
  'type': 'rad'}]

In [56]:
Rvalue1 = 0
for anyR in resistanceList:
  if anyR["name"] != "airGap":
    if anyR["name"] != "woodStud":
      Rvalue1 += anyR["Rvalue"]

In [57]:
Rvalue2 = 0
for anyR in resistanceList:
  if anyR["name"] != "airGap":
    if anyR["name"] != "glassFiber":
      Rvalue2 += anyR["Rvalue"]

In [58]:
print("The unit thermal resistance of isolation section is: " + str(Rvalue1) + " [m^2.degreeC/W]")

The unit thermal resistance of isolation section is: 3.119 [m^2.degreeC/W]


In [59]:
print("The unit thermal resistance of woodstud section is: " + str(Rvalue2) + " [m^2.degreeC/W]")

The unit thermal resistance of woodstud section is: 1.229 [m^2.degreeC/W]


In [60]:
RvalueTot = round(1/(0.75/Rvalue1 + 0.25/Rvalue2) + R8["Rvalue"],4)
print("The overall unit thermal resistance (considering the airgap) is: " + str(RvalueTot) + " [m^2.degreeC/W]")
U_tot = round(1/RvalueTot,4)
print("The overall heat transfer coefficient (considering the airgap) is: " + str(U_tot) + " [W/m^2/degreeC]")

The overall unit thermal resistance (considering the airgap) is: 2.8729 [m^2.degreeC/W]
The overall heat transfer coefficient (considering the airgap) is: 0.3481 [W/m^2/degreeC]


##Calculation of the rate of heat loss through the wall

In [61]:
A_wall = 0.8*50*2.5
deltaT = 22-(-2)
QdotHeatLoss = round(A_wall*U_tot*deltaT,1)
print("The rate of heat loss through the wall is: " + str(QdotHeatLoss) + " [W]")

The rate of heat loss through the wall is: 835.4 [W]


#Part B

##General function for calculating the unit thermal resistance of a multi-layer wall

In [62]:
def effectiveEmissivity (eps1=0.9, eps2=0.9):
  """This function calculate effective emissivity and both epsilon values are defined as 0.9 by default."""
  effectiveEmissivityCalc = round(1/(1/eps1 + 1/eps2 - 1), 2)
  return effectiveEmissivityCalc

In [63]:
def multiLayerRvalue(resistanceList):
  """
  This function, takes separate resistance layers of a multi-layer wall in the form of:
  Ri = {"material":"--", "type":"cond/conv/rad", "length":"--[in mm]", "eps2":"--"} where 'i' is an integer, as its input.
  
  Then, it is able to calculate the unit thermal resistance of each layer, as well as the total unit thermal resistance of the multilayer wall.

  P.S. 1: All the layers, are considered to be in SERIES in this function.
  P.S. 2: Use the material's name, exactly as defined in the materialLibrary.
  P.S. 3: It is not necessary to define the "length", in the case of convective heat transfer with free stream flow.
  P.S. 4: Define the "eps2" if the resistance is radiative. In this case, "eps2" would be the emissivity of the outer surface.
  """
  # Material library. It can be expanded!
  materialLibrary = {
    "gypsumBoard":{"standardLength":13, "Rvalue":0.079},
    "woodFiberBoard":{"standardLength":13, "Rvalue":0.23},
    "woodBevelLappedSiding":{"standardLength":13, "Rvalue":0.14},
    "woodStud":{"standardLength":90, "Rvalue":0.63},
    "glassFiber":{"standardLength":25, "Rvalue":0.7},
    "outsideAir":{"Rvalue":0.03},
    "insideAir":{"Rvalue":0.12} 
  }
  
  # Air gap library. It could be expanded!
  airGapLibrary = {
    20:{0.03:0.65, 0.05:0.61, 0.5:0.25, 0.82:0.18},
    40:{0.03:0.67, 0.05:0.62, 0.5:0.26, 0.82:0.18},
    90:{0.03:0.64, 0.05:0.60, 0.5:0.25, 0.82:0.18}
  }

  #Adding Rvalue of each resistances, to its own dictionary and calculating total unit thermal resistance.
  Rtot=0
  RvalueDic = {}
  i=0
  for anyR in resistanceList:
    i+=1
    if anyR["type"] == "cond":
      anyR["Rvalue"] = round(materialLibrary[anyR["material"]]["Rvalue"] * anyR["length"] / materialLibrary[anyR["material"]]["standardLength"],4)
    elif anyR["type"] == "conv":
      anyR["Rvalue"] = round(materialLibrary[anyR["material"]]["Rvalue"],4)
    elif anyR["type"] == "rad":
      effEm = effectiveEmissivity(eps2=anyR["eps2"])
      anyR["Rvalue"] = round(airGapLibrary[anyR["length"]][effEm],4)
    Rtot += anyR["Rvalue"]
    RvalueDic["Rvalue"+str(i)] = anyR["Rvalue"]

  return print(
      "The unit thermal resistance (R-value) of each layer is: " + str(RvalueDic) + 
      "\n\nThe total unit thermal resistance (R-value) of this multi-layer wall is: " + str(round(Rtot,4)) + " [m^2.degreeC/W]"
      )


##Defining input and calling the function

In [64]:
help(multiLayerRvalue)

Help on function multiLayerRvalue in module __main__:

multiLayerRvalue(resistanceList)
    This function, takes separate resistance layers of a multi-layer wall in the form of:
    Ri = {"material":"--", "type":"cond/conv/rad", "length":"--[in mm]", "eps2":"--"} where 'i' is an integer, as its input.
    
    Then, it is able to calculate the unit thermal resistance of each layer, as well as the total unit thermal resistance of the multilayer wall.
    
    P.S. 1: All the layers, are considered to be in SERIES in this function.
    P.S. 2: Use the material's name, exactly as defined in the materialLibrary.
    P.S. 3: It is not necessary to define the "length", in the case of convective heat transfer with free stream flow.
    P.S. 4: Define the "eps2" if the resistance is radiative. In this case, "eps2" would be the emissivity of the outer surface.



In [65]:
R1 = {"material":"outsideAir", "type":"conv"}
R2 = {"material":"woodBevelLappedSiding", "type":"cond", "length":13}
R3 = {"material":"woodFiberBoard", "type":"cond", "length":13}
R4 = {"material":"glassFiber", "type":"cond", "length":90}
R5 = {"material":"gypsumBoard", "type":"cond", "length":13}
R6 = {"material":"insideAir", "type":"conv"}
R7 = {"material":"airGap", "type":"rad", "length":40, "eps2":0.05}

resistanceList = [R1,R2,R3,R4,R5,R6,R7]    #How can we use a for loop to make the resistance list?
multiLayerRvalue(resistanceList)

The unit thermal resistance (R-value) of each layer is: {'Rvalue1': 0.03, 'Rvalue2': 0.14, 'Rvalue3': 0.23, 'Rvalue4': 2.52, 'Rvalue5': 0.079, 'Rvalue6': 0.12, 'Rvalue7': 0.62}

The total unit thermal resistance (R-value) of this multi-layer wall is: 3.739 [m^2.degreeC/W]
